In [3]:
import findspark
findspark.init("/home/gorazda/spark-2.4.7-bin-hadoop2.7")
import pyspark
from pyspark.sql import SparkSession

In [4]:
spark = SparkSession.builder.appName("linear_regression").getOrCreate()

In [6]:
from pyspark.ml.regression import LinearRegression

In [8]:
training = spark.read.format("libsvm").load("sample_linear_regression_data.txt")

In [9]:
training.show()

+-------------------+--------------------+
|              label|            features|
+-------------------+--------------------+
| -9.490009878824548|(10,[0,1,2,3,4,5,...|
| 0.2577820163584905|(10,[0,1,2,3,4,5,...|
| -4.438869807456516|(10,[0,1,2,3,4,5,...|
|-19.782762789614537|(10,[0,1,2,3,4,5,...|
| -7.966593841555266|(10,[0,1,2,3,4,5,...|
| -7.896274316726144|(10,[0,1,2,3,4,5,...|
| -8.464803554195287|(10,[0,1,2,3,4,5,...|
| 2.1214592666251364|(10,[0,1,2,3,4,5,...|
| 1.0720117616524107|(10,[0,1,2,3,4,5,...|
|-13.772441561702871|(10,[0,1,2,3,4,5,...|
| -5.082010756207233|(10,[0,1,2,3,4,5,...|
|  7.887786536531237|(10,[0,1,2,3,4,5,...|
| 14.323146365332388|(10,[0,1,2,3,4,5,...|
|-20.057482615789212|(10,[0,1,2,3,4,5,...|
|-0.8995693247765151|(10,[0,1,2,3,4,5,...|
| -19.16829262296376|(10,[0,1,2,3,4,5,...|
|  5.601801561245534|(10,[0,1,2,3,4,5,...|
|-3.2256352187273354|(10,[0,1,2,3,4,5,...|
| 1.5299675726687754|(10,[0,1,2,3,4,5,...|
| -0.250102447941961|(10,[0,1,2,3,4,5,...|
+----------

In [10]:
lr = LinearRegression(featuresCol="features", labelCol="label", predictionCol="prediction")

In [11]:
lr_model = lr.fit(training)

In [12]:
lr_model.coefficients

DenseVector([0.0073, 0.8314, -0.8095, 2.4412, 0.5192, 1.1535, -0.2989, -0.5129, -0.6197, 0.6956])

In [13]:
lr_model.intercept

0.14228558260358093

In [14]:
tranning_summary = lr_model.summary

In [15]:
tranning_summary.r2

0.027839179518600154

In [16]:
tranning_summary.rootMeanSquaredError

10.16309157133015

In [17]:
all_data = spark.read.format("libsvm").load("sample_linear_regression_data.txt")

In [20]:
train_data, test_data = all_data.randomSplit([0.7, 0.3])

In [23]:
train_data.describe().show()

+-------+-------------------+
|summary|              label|
+-------+-------------------+
|  count|                358|
|   mean|0.27106686372854627|
| stddev| 10.265908284179737|
|    min|-28.571478869743427|
|    max|  27.78383192005107|
+-------+-------------------+



In [24]:
test_data.describe().show()

+-------+-------------------+
|summary|              label|
+-------+-------------------+
|  count|                143|
|   mean|0.22139414478930142|
| stddev| 10.483147335135351|
|    min|-28.046018037776633|
|    max| 22.647750304177556|
+-------+-------------------+



In [25]:
correct_model = lr.fit(train_data)

In [26]:
test_results = correct_model.evaluate(test_data)

In [28]:
test_results.residuals.show()

+-------------------+
|          residuals|
+-------------------+
| -26.33719062194386|
| -22.48523329524515|
| -20.65617365066641|
|-18.425899988461467|
|-17.156220150955413|
|-17.084334179982747|
| -17.56310995541082|
|-19.168086024411142|
|-17.136618108294776|
|-14.292170222848645|
| -16.77601101491511|
|-18.089047809624162|
|-17.049672443227493|
|-13.565848004611574|
|-18.130853421123632|
|-10.370752973875403|
|-12.860387125069822|
| -11.46077012249686|
| -7.218909935938623|
| -9.986580362583332|
+-------------------+
only showing top 20 rows



In [29]:
test_results.rootMeanSquaredError

10.660733692689789

In [30]:
unlabeled_data = test_data.select("features")

In [31]:
unlabeled_data.show()

+--------------------+
|            features|
+--------------------+
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
+--------------------+
only showing top 20 rows



In [32]:
predictions = correct_model.transform(unlabeled_data)

In [33]:
predictions.show()

+--------------------+-------------------+
|            features|         prediction|
+--------------------+-------------------+
|(10,[0,1,2,3,4,5,...|-1.7088274158327725|
|(10,[0,1,2,3,4,5,...|  2.600672520971725|
|(10,[0,1,2,3,4,5,...| 0.8734108610518749|
|(10,[0,1,2,3,4,5,...|-1.2414186269102545|
|(10,[0,1,2,3,4,5,...|-1.1189934150492162|
|(10,[0,1,2,3,4,5,...|-0.4098661769005972|
|(10,[0,1,2,3,4,5,...|0.13443538447131248|
|(10,[0,1,2,3,4,5,...|  3.444570411362574|
|(10,[0,1,2,3,4,5,...| 1.7770732284620985|
|(10,[0,1,2,3,4,5,...|-1.0567009325306085|
|(10,[0,1,2,3,4,5,...|  1.441243534992769|
|(10,[0,1,2,3,4,5,...| 3.0325648350817294|
|(10,[0,1,2,3,4,5,...| 2.2275195334763045|
|(10,[0,1,2,3,4,5,...|  0.525919940506959|
|(10,[0,1,2,3,4,5,...|  5.208630317753213|
|(10,[0,1,2,3,4,5,...| -2.402474025375794|
|(10,[0,1,2,3,4,5,...|0.38110691361832466|
|(10,[0,1,2,3,4,5,...|-1.0068862585359997|
|(10,[0,1,2,3,4,5,...| -5.192036466901538|
|(10,[0,1,2,3,4,5,...| -2.107770915951927|
+----------